# **Running the BAO fit for a given dataset and settings**

This notebook shows how to run a BAO fit

In [ ]:
%load_ext autoreload
%autoreload 2

## 1. Single BAO fit

In [ ]:
import os
import sys
code_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
save_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
sys.path.append(code_path)
import numpy as np
from scipy.optimize import curve_fit
from utils_data import RedshiftDistributions, GetThetaLimits, WThetaDataCovariance
from utils_baofit import WThetaModelGalaxyBias, WThetaModelBAO, BAOFitInitializer, BAOFit

# helper to select only needed keys for a function
def subset(cfg, *keys):
    return {k: cfg[k] for k in keys}

dataset = "DESY6"
# dataset = "DESY6_dec_below-23.5"
# dataset = "DESY6_dec_above-23.5"
# dataset = "DESY6_DR1tiles_noDESI"
# dataset = "DESY6_DR1tiles_DESIonly"
nz_flag = "fid"

nz = RedshiftDistributions(dataset=dataset, nz_flag=nz_flag, code_path=code_path)
theta_min, theta_max = GetThetaLimits(dataset=dataset, nz_flag=nz_flag, dynamical_theta_limits=False, code_path=code_path).get_theta_limits()

# 1. Arguments
config = {
    "include_wiggles": "y",
    "dataset": dataset,
    "weight_type": 1,
    "mock_id": "mean",  # it will not be used...
    "nz_flag": nz_flag,
    "cov_type": "cosmolike",
    "cosmology_template": "planck_old",
    "cosmology_covariance": "planck",
    "delta_theta": 0.2,
    "theta_min": theta_min,
    "theta_max": theta_max,
    "pow_broadband": [-2, -1, 0],
    "bins_removed": [],
    "diag_only": "n",
    "remove_crosscov": "n",
    "alpha_min": 0.8,
    "alpha_max": 1.2,
    "code_path": code_path,
    "save_path": save_path,
}
config["include_wiggles"] = "" if config["include_wiggles"] == "y" else "_nowiggles"

fit_galaxybias = True

if fit_galaxybias:
    # 2. Load data w(theta) and covariance (for the linear galaxy bias fit)
    wtheta_data_covariance = WThetaDataCovariance(
        **subset(config,
                 "dataset", "weight_type", "nz_flag", "cov_type", "cosmology_covariance", "code_path"),
        mock_id="mean",
        delta_theta=0.2,
        theta_min={bin_z: 0.5 for bin_z in range(nz.nbins)},
        theta_max={bin_z: 2 for bin_z in range(nz.nbins)},
        bins_removed=[],
        diag_only="n",
        remove_crosscov="n",
    )
    theta_data, wtheta_data, cov = wtheta_data_covariance.process()
    wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nz.nbins)])

    # 3. Get the model wtheta (for the linear galaxy bias fit)
    wtheta_model_galaxybias = WThetaModelGalaxyBias(
        **subset(config, "dataset", "nz_flag", "cosmology_template", "save_path"),
        include_wiggles="",  # override for galaxy bias
    )

    # 4. Fit the linear galaxy bias
    popt, pcov = curve_fit(
        wtheta_model_galaxybias.get_wtheta_model(),
        theta_data,
        wtheta_data_concatenated,
        np.ones(nz.nbins),
        bounds=([0] * nz.nbins, [3] * nz.nbins),
        sigma=cov,
        absolute_sigma=False
    )
    galaxy_bias = {bin_z: popt[bin_z] for bin_z in range(len(popt))}

    print(f"Fitted galaxy bias: {[f'bin {bin_z}: {galaxy_bias[bin_z]:.3f}' for bin_z in range(len(galaxy_bias))]}")

else:
    galaxy_bias = {bin_z: 6 for bin_z in range(nz.nbins)}

# 2. Load data w(theta) and covariance
wtheta_data_covariance = WThetaDataCovariance(
    **subset(config,
             "dataset", "weight_type", "mock_id", "nz_flag",
             "cov_type", "cosmology_covariance", "delta_theta",
             "theta_min", "theta_max", "bins_removed", "diag_only",
             "remove_crosscov", "code_path"),
)
theta_data, wtheta_data, cov = wtheta_data_covariance.process()

# 3. Get the model wtheta
wtheta_model = WThetaModelBAO(
    **subset(config, "include_wiggles", "dataset", "nz_flag", "cosmology_template", "pow_broadband", "save_path"),
    galaxy_bias=galaxy_bias,
)

# 4. BAO fit initializer. This basically creates the path to save the results
baofit_initializer = BAOFitInitializer(
    **subset(config, "include_wiggles", "dataset", "weight_type", "mock_id",
             "nz_flag", "cov_type", "cosmology_template", "cosmology_covariance",
             "delta_theta", "theta_min", "theta_max", "pow_broadband",
             "bins_removed", "alpha_min", "alpha_max", "save_path"),
    verbose=True,
)

# 5. Run the BAO fit
baofit = BAOFit(
    baofit_initializer=baofit_initializer,
    wtheta_model=wtheta_model,
    theta_data=theta_data,
    wtheta_data=wtheta_data,
    cov=cov,
    close_fig=False,
    use_multiprocessing=True,
    n_cpu=256,
    overwrite=True,
)
alpha_best, err_alpha, chi2_best, dof = baofit.fit()


## 2. Multiple BAO fits (varying settings)

In [ ]:
import os
import sys
code_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
save_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
sys.path.append(code_path)
import numpy as np
from itertools import product
from scipy.optimize import curve_fit
from utils_data import RedshiftDistributions, GetThetaLimits, WThetaDataCovariance
from utils_baofit import WThetaModelGalaxyBias, WThetaModelBAO, BAOFitInitializer, BAOFit

# helper to select only needed keys for a function
def subset(cfg, *keys):
    return {k: cfg[k] for k in keys}

dataset_list = ["DESY6", "DESY6_dec_below-23.5", "DESY6_dec_above-23.5", "DESY6_DR1tiles_noDESI", "DESY6_DR1tiles_DESIonly"]
nz_flag = "fid"
include_wiggles_list = ["y", "n"]
# include_wiggles_list = ["n"]
weight_type_list = [1, 0]
bins_removed_list = [[], [0], [1], [2], [3], [4], [5], [1, 2, 3, 4, 5], [0, 2, 3, 4, 5], 
                     [0, 1, 3, 4, 5], [0, 1, 2, 4, 5], [0, 1, 2, 3, 5], [0, 1, 2, 3, 4]]
# bins_removed_list = [[]]

for dataset, include_wiggles, weight_type, bins_removed in product(dataset_list, include_wiggles_list, weight_type_list, bins_removed_list):

    nz = RedshiftDistributions(dataset=dataset, nz_flag=nz_flag, code_path=code_path)
    theta_min, theta_max = GetThetaLimits(dataset=dataset, nz_flag=nz_flag, dynamical_theta_limits=False, code_path=code_path).get_theta_limits()
    
    # 1. Arguments
    config = {
        "include_wiggles": include_wiggles,
        "dataset": dataset,
        "weight_type": weight_type,
        "mock_id": "mean",  # it will not be used...
        "nz_flag": "fid",
        "cov_type": "cosmolike",
        "cosmology_template": "planck_old",
        "cosmology_covariance": "planck",
        "delta_theta": 0.2,
        "theta_min": theta_min,
        "theta_max": theta_max,
        "pow_broadband": [-2, -1, 0],
        "bins_removed": bins_removed,
        "diag_only": "n",
        "remove_crosscov": "n",
        "alpha_min": 0.8,
        "alpha_max": 1.2,
        "code_path": code_path,
        "save_path": save_path,
    }
    config["include_wiggles"] = "" if config["include_wiggles"] == "y" else "_nowiggles"
    
    fit_galaxybias = True
    
    if fit_galaxybias:
        # 2. Load data w(theta) and covariance (for the linear galaxy bias fit)
        wtheta_data_covariance = WThetaDataCovariance(
            **subset(config,
                     "dataset", "weight_type", "nz_flag", "cov_type", "cosmology_covariance", "code_path"),
            mock_id="mean",
            delta_theta=0.2,
            theta_min={bin_z: 0.5 for bin_z in range(nz.nbins)},
            theta_max={bin_z: 2 for bin_z in range(nz.nbins)},
            bins_removed=[],
            diag_only="n",
            remove_crosscov="n",
        )
        theta_data, wtheta_data, cov = wtheta_data_covariance.process()
        wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nz.nbins)])
    
        # 3. Get the model wtheta (for the linear galaxy bias fit)
        wtheta_model_galaxybias = WThetaModelGalaxyBias(
            **subset(config, "dataset", "nz_flag", "cosmology_template", "save_path"),
            include_wiggles="",  # override for galaxy bias
        )
    
        # 4. Fit the linear galaxy bias
        popt, pcov = curve_fit(
            wtheta_model_galaxybias.get_wtheta_model(),
            theta_data,
            wtheta_data_concatenated,
            np.ones(nz.nbins),
            bounds=([0] * nz.nbins, [3] * nz.nbins),
            sigma=cov,
            absolute_sigma=False
        )
        galaxy_bias = {bin_z: popt[bin_z] for bin_z in range(len(popt))}
    
        print(f"Fitted galaxy bias: {[f'bin {bin_z}: {galaxy_bias[bin_z]:.3f}' for bin_z in range(len(galaxy_bias))]}")
    
    else:
        galaxy_bias = {bin_z: 6 for bin_z in range(nz.nbins)}
    
    # 2. Load data w(theta) and covariance
    wtheta_data_covariance = WThetaDataCovariance(
        **subset(config,
                 "dataset", "weight_type", "mock_id", "nz_flag",
                 "cov_type", "cosmology_covariance", "delta_theta",
                 "theta_min", "theta_max", "bins_removed", "diag_only",
                 "remove_crosscov", "code_path"),
    )
    theta_data, wtheta_data, cov = wtheta_data_covariance.process()
    
    # 3. Get the model wtheta
    wtheta_model = WThetaModelBAO(
        **subset(config, "include_wiggles", "dataset", "nz_flag", "cosmology_template", "pow_broadband", "save_path"),
        galaxy_bias=galaxy_bias,
    )
    
    # 4. BAO fit initializer. This basically creates the path to save the results
    baofit_initializer = BAOFitInitializer(
        **subset(config, "include_wiggles", "dataset", "weight_type", "mock_id",
                 "nz_flag", "cov_type", "cosmology_template", "cosmology_covariance",
                 "delta_theta", "theta_min", "theta_max", "pow_broadband",
                 "bins_removed", "alpha_min", "alpha_max", "save_path"),
        verbose=True,
    )
    
    # 5. Run the BAO fit
    baofit = BAOFit(
        baofit_initializer=baofit_initializer,
        wtheta_model=wtheta_model,
        theta_data=theta_data,
        wtheta_data=wtheta_data,
        cov=cov,
        close_fig=False,
        use_multiprocessing=True,
        n_cpu=256,
        # overwrite=True,
    )
    alpha_best, err_alpha, chi2_best, dof = baofit.fit()
    

## 3. Multiple BAO fits (DES COLA mocks)

In [ ]:
import os
import sys
code_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
save_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
sys.path.append(code_path)
import numpy as np
from scipy.optimize import curve_fit
from utils_data import RedshiftDistributions, GetThetaLimits, WThetaDataCovariance
from utils_baofit import WThetaModelGalaxyBias, WThetaModelBAO, BAOFitInitializer, BAOFit

# helper to select only needed keys for a function
def subset(cfg, *keys):
    return {k: cfg[k] for k in keys}

dataset = "DESY6_COLA"
# dataset = "DESY6_COLA_dec_below-23.5"
# dataset = "DESY6_COLA_dec_above-23.5"
# dataset = "DESY6_COLA_DR1tiles_noDESI"
# dataset = "DESY6_COLA_DR1tiles_DESIonly"
nz_flag = "mocks"
cov_type = "cosmolike"

n_mocks = 1952

mock_id_list = ["mean"] + list(range(n_mocks))
# mock_id_list = np.arange(0, n_mocks)
# mock_id_list = ["mean"]

nz = RedshiftDistributions(dataset=dataset, nz_flag=nz_flag, code_path=code_path)
theta_min, theta_max = GetThetaLimits(dataset=dataset, nz_flag=nz_flag, dynamical_theta_limits=False, code_path=code_path).get_theta_limits()

for mock_id in mock_id_list:
    # 1. Arguments
    config = {
        "include_wiggles": "y",
        "dataset": dataset,
        "weight_type": 1, # it will not be used...
        "mock_id": mock_id,
        "nz_flag": nz_flag,
        "cov_type": cov_type,
        "cosmology_template": "mice_old",
        "cosmology_covariance": "mice",
        "delta_theta": 0.2,
        "theta_min": theta_min,
        "theta_max": theta_max,
        "pow_broadband": [-2, -1, 0],
        "bins_removed": [],
        "diag_only": "n",
        "remove_crosscov": "n",
        "alpha_min": 0.8,
        "alpha_max": 1.2,
        "code_path": code_path,
        "save_path": save_path,
    }
    config["include_wiggles"] = "" if config["include_wiggles"] == "y" else "_nowiggles"

    if mock_id == "mean":
        close_fig = False
    else:
        close_fig = True
    
    if mock_id == mock_id_list[0]:
        fit_galaxybias = True
    else:
        fit_galaxybias = False
    
    if fit_galaxybias:
        # 2. Load data w(theta) and covariance (for the linear galaxy bias fit)
        wtheta_data_covariance = WThetaDataCovariance(
            **subset(config,
                     "dataset", "weight_type", "nz_flag", "cov_type", "cosmology_covariance", "code_path"),
            mock_id="mean",
            delta_theta=0.2,
            theta_min={bin_z: 0.5 for bin_z in range(nz.nbins)},
            theta_max={bin_z: 2 for bin_z in range(nz.nbins)},
            bins_removed=[],
            diag_only="n",
            remove_crosscov="n",
        )
        theta_data, wtheta_data, cov = wtheta_data_covariance.process()
        wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nz.nbins)])
    
        # 3. Get the model wtheta (for the linear galaxy bias fit)
        wtheta_model_galaxybias = WThetaModelGalaxyBias(
            **subset(config, "dataset", "nz_flag", "cosmology_template", "save_path"),
            include_wiggles="",  # override for galaxy bias
        )
    
        # 4. Fit the linear galaxy bias
        popt, pcov = curve_fit(
            wtheta_model_galaxybias.get_wtheta_model(),
            theta_data,
            wtheta_data_concatenated,
            np.ones(nz.nbins),
            bounds=([0] * nz.nbins, [3] * nz.nbins),
            sigma=cov,
            absolute_sigma=False
        )
        galaxy_bias = {bin_z: popt[bin_z] for bin_z in range(len(popt))}
    
        print(f"Fitted galaxy bias: {[f'bin {bin_z}: {galaxy_bias[bin_z]:.3f}' for bin_z in range(len(galaxy_bias))]}")
    
    # 2. Load data w(theta) and covariance
    wtheta_data_covariance = WThetaDataCovariance(
        **subset(config,
                 "dataset", "weight_type", "mock_id", "nz_flag",
                 "cov_type", "cosmology_covariance", "delta_theta",
                 "theta_min", "theta_max", "bins_removed", "diag_only",
                 "remove_crosscov", "code_path"),
    )
    theta_data, wtheta_data, cov = wtheta_data_covariance.process()
    
    # 3. Get the model wtheta
    wtheta_model = WThetaModelBAO(
        **subset(config, "include_wiggles", "dataset", "nz_flag", "cosmology_template", "pow_broadband", "save_path"),
        galaxy_bias=galaxy_bias,
    )
    
    # 4. BAO fit initializer. This basically creates the path to save the results
    baofit_initializer = BAOFitInitializer(
        **subset(config, "include_wiggles", "dataset", "weight_type", "mock_id",
                 "nz_flag", "cov_type", "cosmology_template", "cosmology_covariance",
                 "delta_theta", "theta_min", "theta_max", "pow_broadband",
                 "bins_removed", "alpha_min", "alpha_max", "save_path"),
        verbose=True,
    )
    
    # 5. Run the BAO fit
    baofit = BAOFit(
        baofit_initializer=baofit_initializer,
        wtheta_model=wtheta_model,
        theta_data=theta_data,
        wtheta_data=wtheta_data,
        cov=cov,
        close_fig=close_fig,
        use_multiprocessing=True,
        n_cpu=256,
        # overwrite=True,
    )
    alpha_best, err_alpha, chi2_best, dof = baofit.fit()


## 4. Multiple BAO fits (DESI EZ/Abacus mocks)

In [ ]:
import os
import sys
code_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
save_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
sys.path.append(code_path)
import numpy as np
from scipy.optimize import curve_fit
from utils_data import RedshiftDistributions, GetThetaLimits, WThetaDataCovariance
from utils_baofit import WThetaModelGalaxyBias, WThetaModelBAO, BAOFitInitializer, BAOFit

# helper to select only needed keys for a function
def subset(cfg, *keys):
    return {k: cfg[k] for k in keys}

delta_z = 0.02

recon = "_recon"
# recon = ""

dataset = f"DESIY1_LRG_EZ_ffa{recon}_deltaz{delta_z}"
# dataset = f"DESIY1_LRG_Abacus_altmtl{recon}_deltaz{delta_z}"
# dataset = f"DESIY1_LRG_EZ_complete{recon}_deltaz{delta_z}"
# dataset = f"DESIY1_LRG_Abacus_complete{recon}_deltaz{delta_z}"
nz_flag = "mocks"

if "EZ" in dataset:
    n_mocks = 1000
elif "Abacus" in dataset:
    n_mocks = 25

# mock_id_list = ["mean"] + list(range(n_mocks))
# mock_id_list = np.arange(0, n_mocks)
mock_id_list = ["mean"]

if delta_z == 0.028:
    bins_removed = []
    # bins_removed = list(map(int, np.arange(7, 25)))  # LRG1
    # bins_removed = list(map(int, np.concatenate([np.arange(0, 7), np.arange(14, 25)])))  # LRG2
    # bins_removed = list(map(int, np.arange(0, 14)))  # LRG3
elif delta_z == 0.02:
    bins_removed = []
    # bins_removed = list(map(int, np.arange(10, 35)))  # LRG1
    # bins_removed = list(map(int, np.concatenate([np.arange(0, 10), np.arange(20, 35)])))  # LRG2
    # bins_removed = list(map(int, np.arange(0, 20)))  # LRG3

nz = RedshiftDistributions(dataset=dataset, nz_flag=nz_flag, code_path=code_path)
theta_min, theta_max = GetThetaLimits(dataset=dataset, nz_flag=nz_flag, dynamical_theta_limits=True, code_path=code_path).get_theta_limits()

for mock_id in mock_id_list:
    # 1. Arguments
    config = {
        "include_wiggles": "y",
        "dataset": dataset,
        "weight_type": 1, # it will not be used...
        "mock_id": mock_id,
        "nz_flag": nz_flag,
        "cov_type": "mocks",
        "cosmology_template": "desifid",
        "cosmology_covariance": "desifid",
        "delta_theta": 0.2,
        "theta_min": theta_min,
        "theta_max": theta_max,
        "pow_broadband": [-1, 0],
        "bins_removed": bins_removed,
        "diag_only": "n",
        "remove_crosscov": "n",
        "alpha_min": 0.8,
        "alpha_max": 1.2,
        "code_path": code_path,
        "save_path": save_path,
    }
    config["include_wiggles"] = "" if config["include_wiggles"] == "y" else "_nowiggles"

    if mock_id == "mean":
        close_fig = False
    else:
        close_fig = True
    
    if mock_id == mock_id_list[0]:
        fit_galaxybias = True
    else:
        fit_galaxybias = False
    
    if fit_galaxybias:
        # 2. Load data w(theta) and covariance (for the linear galaxy bias fit)
        wtheta_data_covariance = WThetaDataCovariance(
            **subset(config,
                     "dataset", "weight_type", "nz_flag", "cov_type", "cosmology_covariance", "code_path"),
            mock_id="mean",
            delta_theta=0.2,
            theta_min={bin_z: 0.5 for bin_z in range(nz.nbins)},
            theta_max={bin_z: 2 for bin_z in range(nz.nbins)},
            bins_removed=[],
            diag_only="n",
            remove_crosscov="n",
        )
        theta_data, wtheta_data, cov = wtheta_data_covariance.process()
        wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nz.nbins)])
    
        # 3. Get the model wtheta (for the linear galaxy bias fit)
        wtheta_model_galaxybias = WThetaModelGalaxyBias(
            **subset(config, "dataset", "nz_flag", "cosmology_template", "save_path"),
            include_wiggles="",  # override for galaxy bias
        )
    
        # 4. Fit the linear galaxy bias
        popt, pcov = curve_fit(
            wtheta_model_galaxybias.get_wtheta_model(),
            theta_data,
            wtheta_data_concatenated,
            np.ones(nz.nbins),
            bounds=([0] * nz.nbins, [3] * nz.nbins),
            sigma=cov,
            absolute_sigma=False
        )
        galaxy_bias = {bin_z: popt[bin_z] for bin_z in range(len(popt))}
    
        print(f"Fitted galaxy bias: {[f'bin {bin_z}: {galaxy_bias[bin_z]:.3f}' for bin_z in range(len(galaxy_bias))]}")
    
    # 2. Load data w(theta) and covariance
    wtheta_data_covariance = WThetaDataCovariance(
        **subset(config,
                 "dataset", "weight_type", "mock_id", "nz_flag",
                 "cov_type", "cosmology_covariance", "delta_theta",
                 "theta_min", "theta_max", "bins_removed", "diag_only",
                 "remove_crosscov", "code_path"),
    )
    theta_data, wtheta_data, cov = wtheta_data_covariance.process()
    
    # 3. Get the model wtheta
    wtheta_model = WThetaModelBAO(
        **subset(config, "include_wiggles", "dataset", "nz_flag", "cosmology_template", "pow_broadband", "save_path"),
        galaxy_bias=galaxy_bias,
    )
    
    # 4. BAO fit initializer. This basically creates the path to save the results
    baofit_initializer = BAOFitInitializer(
        **subset(config, "include_wiggles", "dataset", "weight_type", "mock_id",
                 "nz_flag", "cov_type", "cosmology_template", "cosmology_covariance",
                 "delta_theta", "theta_min", "theta_max", "pow_broadband",
                 "bins_removed", "alpha_min", "alpha_max", "save_path"),
        verbose=True,
    )
    
    # 5. Run the BAO fit
    baofit = BAOFit(
        baofit_initializer=baofit_initializer,
        wtheta_model=wtheta_model,
        theta_data=theta_data,
        wtheta_data=wtheta_data,
        cov=cov,
        close_fig=close_fig,
        use_multiprocessing=True,
        n_cpu=256,
        # overwrite=True,
    )
    alpha_best, err_alpha, chi2_best, dof = baofit.fit()
